In [ ]:
import torch
import albumentations as A
from glob import glob
from tqdm import tqdm
from utils import read_image
from utils import read_mask
from utils import testing_model
from utils import bfs
import os
from pathlib import Path
import numpy as np
import cv2

img_shape = [512, 512]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
unet_path = 'trained_models/fold 1/model_resnet18_lovasz_05_05_120.pt'

def attention_masks_for_images(unet_path, images_dir, dst_dir):
    model = torch.load(unet_path, map_location=device)
    model.eval()
    
    if not Path(dst_dir).exists():
        os.mkdir(dst_dir)
    
    for image_path in tqdm(glob(images_dir + '/*')):
        new_image = np.zeros((512, 512, 3))
        image = read_image(image_path)
        attention_mask, _ = testing_model(model, image)
        for row in range(512):
            for col in range(512):
                new_image[row, col, :] = (255, 255, 255) if attention_mask[row, col] > 0 else image[row, col, :]
        
        cv2.imwrite(dst_dir + '/' + image_path.split('/')[-1], new_image)

def convert_masks_to_yolo(images_path, masks_path, dst_path):
    model = torch.load('type.pt', map_location=device)
    if not Path(dst_path).exists():
        os.mkdir(dst_path)
        
    for image_path in tqdm(images_path):
        image = read_image(image_path)
        mask = read_mask('/'.join(masks_path[0].split('/')[:-1]) + '/' + 
                         image_path.split('/')[-1])
        timage = transform(image=image)['image']
        timage = torch.tensor(timage).permute(2, 0, 1).view(1, 3, img_shape[0], img_shape[1])
        timage = timage.to(device)
        cls = -1
        with torch.no_grad():
            cls = model(timage).softmax(dim=1).argmax(dim=1).detach().cpu().numpy()[0]
            
        comps = bfs(mask)
        bboxes = []
        for comp in comps:
            min_y, max_y, min_x, max_x = 10000000, -1, 10000000, -1
            for node in comp[0]:
                row, col = node // mask.shape[1], node % mask.shape[1]
                min_y, min_x, max_y, max_x = min(min_y, row), min(min_x, col), max(max_y, row), max(max_x, col)
            heigth, width = max_y - min_y - 2, max_x - min_x - 2
            center_x, center_y = (max_x + min_x) // 2, (max_y + min_y) // 2
            
            assert (center_x - width / 2 >= 0.0), 'x- fucked'
            assert (center_x + width / 2 <= mask.shape[1]), 'x+ fucked'
            assert (center_y - heigth / 2 >= 0.0), 'y- fucked'
            assert (center_y + heigth / 2 <= mask.shape[0]), 'y+ fucked'
            
            bboxes.append([center_x / mask.shape[1], center_y / mask.shape[0],
                           width / mask.shape[1], heigth / mask.shape[0]])
        
        with open(os.path.join(dst_path, image_path.split('/')[-1][:-4] + '.txt'), 'w') as file:
            for bbox in bboxes:
                print(str(cls), ' '.join(list(map(str, bbox))), file=file)    

In [ ]:
attention_masks_for_images(unet_path, 'new_images', 'attention_images')

In [ ]:
images_path = glob('attention_images/*')
masks_path = glob('new_masks/*')
convert_masks_to_yolo(images_path, masks_path, 'labels')

In [ ]:
from sklearn.model_selection import train_test_split

def create_yolo_dataset(images_dir, labels_dir, dst_dir):
    if not Path(dst_dir).exists():
        os.mkdir(dst_dir)
        
    images_path = list(sorted(glob(images_dir + '/*')))
    labels_path = list(sorted(glob(labels_dir + '/*')))
    
    images_path_train, images_path_valid, labels_path_train, labels_path_valid = train_test_split(
        images_path, labels_path, test_size=0.2, random_state=69
    )
    
    if not Path(dst_dir + '/train').exists():
        os.mkdir(dst_dir + '/train')
    
    if not Path(dst_dir + '/valid').exists():
        os.mkdir(dst_dir + '/valid')
        
    if not Path(dst_dir + '/train/images').exists():
        os.mkdir(dst_dir + '/train/images')
    
    if not Path(dst_dir + '/valid/images').exists():
        os.mkdir(dst_dir + '/valid/images')
    
    if not Path(dst_dir + '/train/labels').exists():
        os.mkdir(dst_dir + '/train/labels')
    
    if not Path(dst_dir + '/valid/labels').exists():
        os.mkdir(dst_dir + '/valid/labels')
        
    for image_path, label_path in zip(images_path_train, labels_path_train):
        os.system(f'cp {image_path} {dst_dir}/train/images/{image_path.split("/")[-1]}')
        os.system(f'cp {label_path} {dst_dir}/train/labels/{label_path.split("/")[-1]}')
        
    for image_path, label_path in zip(images_path_valid, labels_path_valid):
        os.system(f'cp {image_path} {dst_dir}/valid/images/{image_path.split("/")[-1]}')
        os.system(f'cp {label_path} {dst_dir}/valid/labels/{label_path.split("/")[-1]}')


In [ ]:
create_yolo_dataset('attention_images', 'labels', 'attention_ds')

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')
model.train(data='/home/anton/PycharmProjects/kaggle_competitions/severstal_aiijc/attention_ds/data.yaml',
            epochs=100, batch=8)

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')

# model.train(data='/home/anton/PycharmProjects/kaggle_competitions/severstal_aiijc/yolo_dataset_cs000_01/yolo_dataset_cs000_01/data.yaml',
#             epochs=100, cos_lr=True, dropout=0.2, mixup=0.2, batch=8)
# model.train(data='/home/anton/PycharmProjects/kaggle_competitions/severstal_aiijc/10k_dataset/10k_dataset/data.yaml',
#             epochs=100, batch=8, single_cls=True)
model.train(data='/home/anton/PycharmProjects/kaggle_competitions/severstal_aiijc/yolo_dataset_cs000_02/yolo_dataset_cs000_02/data.yaml',
            epochs=100, batch=8, box=4.0)

# Швелера

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')

# model.train(data='/home/anton/PycharmProjects/kaggle_competitions/severstal_aiijc/yolo_dataset_cs000_01/yolo_dataset_cs000_01/data.yaml',
#             epochs=100, cos_lr=True, dropout=0.2, mixup=0.2, batch=8)
# model.train(data='/home/anton/PycharmProjects/kaggle_competitions/severstal_aiijc/10k_dataset/10k_dataset/data.yaml',
#             epochs=100, batch=8, single_cls=True)
model.train(data='/home/anton/PycharmProjects/kaggle_competitions/severstal_aiijc/channel_pipes_detection_aug/channel_pipes_detection_aug/data.yaml',
            epochs=100, batch=8, box=14.0)

# Segmentation yolo

In [ ]:
#segmentation yolo
from pathlib import Path
from glob import glob
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
from PIL import Image
import os
import albumentations as A
from collections import deque
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt
from utils import read_image
from utils import read_mask
from utils import bfs
from utils import sort_points_clockwise

transforms_val = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=0.5, std=0.5)
])
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torch.load('type.pt', map_location=device)
model.eval()

if not Path('yolo_seg_dataset').exists():
    os.mkdir('yolo_seg_dataset')
    
if not Path('yolo_seg_dataset/train').exists():
    os.mkdir('yolo_seg_dataset/train')
    
if not Path('yolo_seg_dataset/test').exists():
    os.mkdir('yolo_seg_dataset/test')
    
if not Path('yolo_seg_dataset/valid').exists():
    os.mkdir('yolo_seg_dataset/valid')
    
if not Path('yolo_seg_dataset/train/images').exists():
    os.mkdir('yolo_seg_dataset/train/images')
    
if not Path('yolo_seg_dataset/test/images').exists():
    os.mkdir('yolo_seg_dataset/test/images')
    
if not Path('yolo_seg_dataset/valid/images').exists():
    os.mkdir('yolo_seg_dataset/valid/images')
    
if not Path('yolo_seg_dataset/train/labels').exists():
    os.mkdir('yolo_seg_dataset/train/labels')
    
if not Path('yolo_seg_dataset/test/labels').exists():
    os.mkdir('yolo_seg_dataset/test/labels')
    
if not Path('yolo_seg_dataset/valid/labels').exists():
    os.mkdir('yolo_seg_dataset/valid/labels')
    
images_path = list(sorted(glob('new_images/*')))
masks_path = list(sorted(glob('new_masks/*')))


images_path_train, images_path_test, masks_path_train, masks_path_test = train_test_split(
    images_path, masks_path, test_size=0.25
)

images_path_test, images_path_val, masks_path_test, masks_path_val = train_test_split(
    images_path_test, masks_path_test, test_size=0.4
)
def process_folder_for_yolo_seg(images_path, masks_path, mode):
    i = 0
    for image_path, mask_path in tqdm(zip(images_path, masks_path), total=len(images_path),
                                     position=0, leave=True):
        image = read_image(image_path)
        mask = read_mask(mask_path)
        comps = bfs(mask)
        
        assert len(comps) > 0, 'no comps wtf'

        timage = transforms_val(image=image)['image']
        timage = torch.tensor(timage).to(device)
        timage = timage.permute(2, 0, 1)
        timage = timage.view(1, *timage.shape)

        cls = model(timage).softmax(dim=1).argmax(dim=1).detach().cpu().numpy()[0]
        total = []
        for comp in comps:
            visited, degs = comp
            
            border_nodes = []
            for node_idx in range(len(visited)):
                if degs[node_idx] != 4:
                    border_nodes.append(visited[node_idx])
    
            assert len(border_nodes) > 0, 'addd'
            
            coords = [((node % mask.shape[1]) / mask.shape[1], (node // mask.shape[1]) / mask.shape[0])
                  for node in border_nodes]
            
            
            assert len(coords) > 0, 'too little'
            sorted_coords = sort_points_clockwise(coords)
            sorted_coords = [sorted_coords[i] for i in range(0, len(sorted_coords), 
                                                    int(len(sorted_coords) ** (1/2)))]
            
            ss = []
            for pair in sorted_coords:
                ss.append(pair[0]), ss.append(pair[1])
            total.append(ss)
        
        if i == 0:
            bordered_mask = np.zeros((512, 512))
            for tt in total:
                for j in range(0, len(tt), 2):
                    x, y = int(tt[j] * mask.shape[1]), int(tt[j + 1] * mask.shape[0])
                    bordered_mask[y, x] = 255
            cv2.imwrite('bordered_mask.png', bordered_mask)
            i += 1
        
        with open(f'yolo_seg_dataset/{mode}/labels/' + mask_path.split('/')[-1][:-4] + '.txt', 'w') as file:
            for pack in total:
                print(str(cls), ' '.join(map(str, pack)), file=file)

        cv2.imwrite(f'yolo_seg_dataset/{mode}/images/' + image_path.split('/')[-1], image)
    
process_folder_for_yolo_seg(images_path_train, masks_path_train, 'train')
process_folder_for_yolo_seg(images_path_test, masks_path_test, 'test')
process_folder_for_yolo_seg(images_path_val, masks_path_val, 'valid')

    

In [ ]:
from ultralytics import YOLO
model = YOLO('yolo11n-seg.pt')
model.train(data='/home/anton/PycharmProjects/kaggle_competitions/severstal_aiijc/shvellers_seg/data.yaml', 
           epochs=200, batch=8)

In [ ]:
!yolo task=segment mode=train model=yolo11n-seg.pt data=/home/anton/PycharmProjects/kaggle_competitions/severstal_aiijc/ugols_seg/data.yaml epochs=200 imgsz=640 plots=True

In [ ]:
from ultralytics import YOLO
model = YOLO('yolo11n-seg.pt')
model.train(data='/home/anton/PycharmProjects/kaggle_competitions/severstal_aiijc/shvellers_seg/data.yaml', 
           epochs=200, batch=8)

# One model for all types of pipes detection

In [ ]:
import torch
import albumentations as A
from glob import glob
from tqdm import tqdm
from utils import read_image
from utils import read_mask
from utils import testing_model
from utils import bfs
import os
from pathlib import Path
import numpy as np
import cv2
from ultralytics import YOLO
from sklearn.model_selection import train_test_split

In [ ]:
merged_ds_path = 'merged_dataset'
if not Path(merged_ds_path).exists():
    os.mkdir(merged_ds_path)
    
square_and_circles_ds_path = 'new_yolo_seg_dataset'
ugols_ds_path = 'new_ugols_seg_aug'
shvellers_ds_path = 'new_shvellers_seg_aug'

sqc_images_path = list(sorted(glob(os.path.join(square_and_circles_ds_path, 'images') + '/*')))
sqc_labels_path = list(sorted(glob(os.path.join(square_and_circles_ds_path, 'labels') + '/*')))

sqc_images_path = [p for p in sqc_images_path if int(p.split('/')[-1].split('.')[0]) < 186]
sqc_labels_path = [p for p in sqc_labels_path if int(p.split('/')[-1].split('.')[0]) < 186]
sqc_images_path_add = [p for p in sqc_images_path if int(p.split('/')[-1].split('.')[0]) >= 186]
sqc_labels_path_add = [p for p in sqc_labels_path if int(p.split('/')[-1].split('.')[0]) >= 186]
# sqc_images_path = list(sorted(glob('original_images' + '/*')))
# sqc_labels_path = list(sorted(glob(os.path.join(square_and_circles_ds_path, 'labels') + '/*')))
# sqc_labels_path = [p for p in sqc_labels_path if 'original_images/' + p.split('/')[-1].split('.')[0] + '.png' in sqc_images_path]

sqc_images_path_train, sqc_images_path_val, sqc_labels_path_train, sqc_labels_path_val = train_test_split(
    sqc_images_path, sqc_labels_path, test_size=0.1, random_state=69
)
print(len(sqc_images_path_train))

if not Path(os.path.join(merged_ds_path, 'train')).exists():
    os.mkdir(os.path.join(merged_ds_path, 'train'))
    
if not Path(os.path.join(merged_ds_path, 'valid')).exists():
    os.mkdir(os.path.join(merged_ds_path, 'valid'))
    
if not Path(os.path.join(merged_ds_path, 'train', 'images')).exists():
    os.mkdir(os.path.join(merged_ds_path, 'train', 'images'))
    
if not Path(os.path.join(merged_ds_path, 'valid', 'images')).exists():
    os.mkdir(os.path.join(merged_ds_path, 'valid', 'images'))
    
if not Path(os.path.join(merged_ds_path, 'train', 'labels')).exists():
    os.mkdir(os.path.join(merged_ds_path, 'train', 'labels'))
    
if not Path(os.path.join(merged_ds_path, 'valid', 'labels')).exists():
    os.mkdir(os.path.join(merged_ds_path, 'valid', 'labels'))
    
for image_path, label_path in zip(sqc_images_path_train, sqc_labels_path_train):
    os.system(f"cp {image_path} {merged_ds_path + '/train/images/' + image_path.split('/')[-1]} ")
    os.system(f"cp {label_path} {merged_ds_path + '/train/labels/' + label_path.split('/')[-1].split('.')[0] + '.txt'} ")
    
for image_path, label_path in zip(sqc_images_path_val, sqc_labels_path_val):
    os.system(f"cp {image_path} {merged_ds_path + '/valid/images/' + image_path.split('/')[-1]} ")
    os.system(f"cp {label_path} {merged_ds_path + '/valid/labels/' + label_path.split('/')[-1].split('.')[0] + '.txt'} ")
    
for image_path, label_path in zip(sqc_images_path_add, sqc_labels_path_add):
    os.system(f"cp {image_path} {merged_ds_path + '/train/images/' + image_path.split('/')[-1]} ")
    os.system(f"cp {label_path} {merged_ds_path + '/train/labels/' + label_path.split('/')[-1].split('.')[0] + '.txt'} ")
    
shv_images_path_train = list(sorted(glob(os.path.join(shvellers_ds_path, 'train', 'images') + '/*')))
shv_images_path_val = list(sorted(glob(os.path.join(shvellers_ds_path, 'valid', 'images') + '/*')))
shv_labels_path_train = list(sorted(glob(os.path.join(shvellers_ds_path, 'train', 'labels') + '/*')))
shv_labels_path_val = list(sorted(glob(os.path.join(shvellers_ds_path, 'valid', 'labels') + '/*')))


for image_path, label_path in zip(shv_images_path_train, shv_labels_path_train):
    os.system(f"cp {image_path} {merged_ds_path + '/train/images/' + image_path.split('/')[-1]} ")
    os.system(f"cp {label_path} {merged_ds_path + '/train/labels/' + label_path.split('/')[-1].split('.')[0] + '.txt'} ")
    
for image_path, label_path in zip(shv_images_path_val, shv_labels_path_val):
    os.system(f"cp {image_path} {merged_ds_path + '/valid/images/' + image_path.split('/')[-1]} ")
    os.system(f"cp {label_path} {merged_ds_path + '/valid/labels/' + label_path.split('/')[-1].split('.')[0] + '.txt'} ")
    
ugols_images_path_train = list(sorted(glob(os.path.join(ugols_ds_path, 'train', 'images') + '/*')))
ugols_images_path_val = list(sorted(glob(os.path.join(ugols_ds_path, 'valid', 'images') + '/*')))
ugols_labels_path_train = list(sorted(glob(os.path.join(ugols_ds_path, 'train', 'labels') + '/*')))
ugols_labels_path_val = list(sorted(glob(os.path.join(ugols_ds_path, 'valid', 'labels') + '/*')))
print()

for image_path, label_path in zip(ugols_images_path_train, ugols_labels_path_train):
    os.system(f"cp {image_path} {merged_ds_path + '/train/images/' + 'cc' + image_path.split('/')[-1]} ")
    os.system(f"cp {label_path} {merged_ds_path + '/train/labels/' + 'cc' + label_path.split('/')[-1].split('.')[0] + '.txt'} ")
    
for image_path, label_path in zip(ugols_images_path_val, ugols_labels_path_val):
    os.system(f"cp {image_path} {merged_ds_path + '/valid/images/' + 'cc' + image_path.split('/')[-1]} ")
    os.system(f"cp {label_path} {merged_ds_path + '/valid/labels/' + 'cc' + label_path.split('/')[-1].split('.')[0] + '.txt'} ")
    
os.system(f"cp data.yaml {merged_ds_path}/data.yaml")

In [ ]:
from ultralytics import YOLO
model = YOLO('yolo11n.pt')
model.train(data='/home/anton/PycharmProjects/kaggle_competitions/severstal_aiijc/merged_dataset/data.yaml', 
           epochs=200, batch=16, box=15.0, cls=1.5)

In [ ]:
model.export(format='torchscript')

In [ ]:
import torch
torch_script_model = torch.jit.load('runs/detect/train39/weights/best.torchscript')


In [ ]:
output = torch_script_model('')